# Building a spam filter with Naive Bayes

This project aims to build a spam filter for SMS messages using the [multinomial Naive Bayes algorithm]( https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_na%C3%AFve_Bayes). We will use a dataset put together by Tiago A. Almeida and José María Gómez Hidalgo. The dataset can be downloaded from the [The UCI Machine Learning Repository]( https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more detail on [this page]( http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers. The spam filter needs to classify new messages with an accuracy greater than 80%.

## Overview of the data

In [1]:
# Import modules
import numpy as np
import pandas as pd
import re

In [2]:
# Import the dataset
labelled_sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
labelled_sms.sample(10, random_state=0)

,Label,SMS
4456,ham,"Storming msg: Wen u lift d phne, u say ""HELLO""..."
690,spam,<Forwarded from 448712404000>Please CALL 08712...
944,ham,And also I've sorta blown him off a couple tim...
3768,ham,"Sir Goodmorning, Once free call me."
1189,ham,All will come alive.better correct any good lo...
4437,ham,"House-Maid is the murderer, coz the man was mu..."
3587,spam,I am hot n horny and willing I live local to y...
1982,ham,"Sorry, I'll call later in meeting any thing re..."
2038,ham,Oh sorry please its over
2078,ham,Hey hun-onbus goin 2 meet him. He wants 2go ou...


In [3]:
print("This dataset has {} rows.".format(labelled_sms.shape[0]))

This dataset has 5572 rows.


In [4]:
labelled_sms["Label"].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

The label `ham` means not-spam. The dataset contains 13.4% of spam messages.

## Data cleaning

The spam algorithm we are developing will not account for the punctuation. We will remove all non-alphanumeric characters. The algorithm will also interpret upper and lowercase letters as being the same. We will transform all letters to lowercase.


In [5]:
labelled_sms["SMS"] = labelled_sms["SMS"].str.replace(r"\W", " ").str.lower().str.split()
labelled_sms.sample(10, random_state=0)

,Label,SMS
4456,ham,"[storming, msg, wen, u, lift, d, phne, u, say,..."
690,spam,"[forwarded, from, 448712404000, please, call, ..."
944,ham,"[and, also, i, ve, sorta, blown, him, off, a, ..."
3768,ham,"[sir, goodmorning, once, free, call, me]"
1189,ham,"[all, will, come, alive, better, correct, any,..."
4437,ham,"[house, maid, is, the, murderer, coz, the, man..."
3587,spam,"[i, am, hot, n, horny, and, willing, i, live, ..."
1982,ham,"[sorry, i, ll, call, later, in, meeting, any, ..."
2038,ham,"[oh, sorry, please, its, over]"
2078,ham,"[hey, hun, onbus, goin, 2, meet, him, he, want..."


## Training and test sets

To test the spam filter, we will split our dataset into two categories:

* Training set - used to train the algorithm on how to classify messages. This set will have 4,458 messages (about 80% of the dataset).
* Test set -  Used to test how good the spam filter is for classifying new messages. This set will have 1,114 messages (about 20% of the dataset).

In [6]:
# Randomize the order of the dataset
labelled_sms = labelled_sms.sample(frac=1, random_state=1)

# Split into training and test sets
training_set = labelled_sms[:4458].reset_index(drop=True)
test_set = labelled_sms[-1114:].reset_index(drop=True)

# Check the percentages of spam emails on both sets
print("Training set has {:.1f}% of spam emails.".format(training_set["Label"].value_counts(normalize=True)["spam"] * 100))
print("Test set has {:.1f}% of spam emails.".format(test_set["Label"].value_counts(normalize=True)["spam"] * 100))

Training set has 13.5% of spam emails.
Test set has 13.2% of spam emails.


The percentage of spam emails on both subsets are similar to that of the original set.

## Vocabulary and word count

Now we will create a list with all the unique words that occur in the messages of our training set. After, we will count the number of times that each unique word appears in each message.

In [7]:
# List with all the unique words
vocabulary = list(set([j for i in training_set["SMS"] for j in i]))

print("Vocabulary has {} unique words.".format(len(vocabulary)))

# Count the number of times that each unique word appears in each message
word_count = {word: [0] * len(training_set["SMS"]) for word in vocabulary}

for index, sms in enumerate(training_set["SMS"]):

    for word in sms:
       
        word_count[word][index] += 1

# Transform word_count into a DataFrame 
word_count = pd.DataFrame(word_count)

# Concatenate our training set with our word count
training_set = pd.concat([training_set, word_count], axis=1)
training_set.head()

Vocabulary has 7783 unique words.


,Label,SMS,desparate,inde,feathery,messed,545,archive,happier,rearrange,...,heltini,planning,choose,24th,tightly,wicket,bash,watevr,word,creep
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Initialize data

The Naive Bayes algorithm that we will implement in our spam filter uses the following equations:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

where

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

We will use Laplace smoothing $\alpha = 1$ and will start by calculating the constant terms of these equations.

In [8]:
alpha = 1

is_spam = training_set["Label"] == "spam"
p_spam = (is_spam).sum() / training_set.shape[0]
print("p_spam = {:.3f}".format(p_spam))

p_ham = 1 - p_spam
print("p_ham = {:.3f}".format(p_ham))

n_spam = training_set.iloc[is_spam.values,2:].to_numpy().sum()
print("n_spam = {}".format(n_spam))

n_ham = training_set.iloc[~is_spam.values,2:].to_numpy().sum()
print("n_ham = {}".format(n_ham))

n_vocab = len(vocabulary)
print("n_vocab = {}".format(n_vocab))

p_wi_given_spam = {word: (training_set.loc[is_spam,word].sum() + alpha) / (n_spam + alpha * n_vocab) for word in vocabulary}

p_wi_given_ham = {word: (training_set.loc[~is_spam,word].sum() + alpha) / (n_ham + alpha * n_vocab) for word in vocabulary}

p_spam = 0.135
p_ham = 0.865
n_spam = 15190
n_ham = 57237
n_vocab = 7783


If $wi$ it is a new word, the parameters $P(w_i|Spam)$ and $P(w_i|Ham) become:

\begin{equation}
P(w_i|Spam) = \frac{\alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{\alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [9]:
p_new_wi_given_spam = alpha / (n_spam + alpha * n_vocab)
print("p_new_wi_given_spam = {:.3E}".format(p_new_wi_given_spam))

p_new_wi_given_ham = alpha / (n_ham + alpha * n_vocab)
print("p_new_wi_given_ham = {:.3E}".format(p_new_wi_given_ham))

p_new_wi_given_spam = 4.353E-05
p_new_wi_given_ham = 1.538E-05


In [10]:
word = "money"
print(p_wi_given_spam[word])
print(p_wi_given_ham[word])

0.00021764680276846734
0.0006920947400799754


## Classifying a new message

Now that we've initialized all the data, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
* Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
* Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
  * If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
  * If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
  * If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.

In [11]:
def classify(sms):

    # Check if it is a string
    if type(sms) == str: sms = re.sub('\W', ' ', sms).lower().split()
    
    # Calculate p_spam_given_sms and p_ham_given_sms
    p_spam_given_sms = p_spam * np.prod([p_wi_given_spam[word] if word in p_wi_given_spam else 1 for word in sms])
    p_ham_given_sms = p_ham * np.prod([p_wi_given_ham[word] if word in p_wi_given_ham else 1 for word in sms])

    # Classify the message
    if p_ham_given_sms > p_spam_given_sms:
        label = "ham"
    elif p_ham_given_sms < p_spam_given_sms:
        label = "spam"
    else:
        label = "unclassified"
        
    return label, p_spam_given_sms, p_ham_given_sms

In [12]:
label, p_spam_given_sms, p_ham_given_sms = classify("WINNER!! This is the secret code to unlock the money: C3421.")
print('Label:', label)
print('P(Spam|sms):', p_spam_given_sms)
print('P(Ham|sms):', p_ham_given_sms)

Label: spam
P(Spam|sms): 1.3481290211300841e-25
P(Ham|sms): 1.936804902858988e-27


In [13]:
label, p_spam_given_sms, p_ham_given_sms = classify("Sounds good, Tom, then see u there")
print('Label:', label)
print('P(Spam|sms):', p_spam_given_sms)
print('P(Ham|sms):', p_ham_given_sms)

Label: ham
P(Spam|sms): 2.4372375665888113e-25
P(Ham|sms): 3.687530435009238e-21


## Measuring the spam filter's accuracy

We'll now try to determine the accuracy of the spam filter using our test set of 1,114 messages.

In [14]:
test_set['predicted'] = test_set['SMS'].apply(lambda x: classify(x)[0])
test_set.head()

,Label,SMS,predicted
0,ham,"[later, i, guess, i, needa, do, mcat, study, too]",ham
1,ham,"[but, i, haf, enuff, space, got, like, 4, mb]",ham
2,spam,"[had, your, mobile, 10, mths, update, to, late...",spam
3,ham,"[all, sounds, good, fingers, makes, it, diffic...",ham
4,ham,"[all, done, all, handed, in, don, t, know, if,...",ham


In [15]:
"Spam accuracy: {:.1f}%".format((test_set['Label'] == test_set['predicted']).sum() / test_set.shape[0] * 100)

'Spam accuracy: 98.7%'

## Conclusions and further developments

We managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%.
In terms of further developments, we could isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions. Also, we might be able to increase the accuracy of the filtering process by accounting for letter case and punctuation.